In [1]:
#Install google trend API first If you don't have it installed already
#!pip install pytrends 

import pandas as pd

from pytrends.request import TrendReq
pytrend = TrendReq()

# importing module
import time

In [2]:
#saerch_index class gets the monthly search data from google trend using the google tred API
class search_index:

    def __init__(self, word_search, location):
        self.word_search = word_search
        self.location = location

    def df_search_tiator(self):
        
        word_search = self.word_search
        geo_code_province = self.location

        #Creating an empty data frame to add search data
        df_search_main = pd.DataFrame()    
        row_length, col_length = df_search_main.shape 

        for geo_code_index in geo_code_province:
            
            #If the user have list of their own word search 
            #otherwise it uses the google search recomendation
            if isinstance(word_search, list) == True:
                search_queries = word_search

            else:
                #The initial word search
                pytrend.build_payload(kw_list=[word_search], timeframe='all', geo=geo_code_index)
                search_queries = pytrend.related_queries()[word_search]['top']['query'] #this gets the related serch words

                #the data for the initial search
                over_time_search = pytrend.interest_over_time()

                #Inserting data into dataframe
                df_search_main.insert(col_length, (geo_code_index +'_'+ word_search), over_time_search[word_search], True)
                row_length, col_length = df_search_main.shape
            

            #Loops through the search words
            for rel_query in search_queries:

                pytrend.build_payload(kw_list=[rel_query], timeframe='all', geo=geo_code_index)
                rel_query_over_time_search = pytrend.interest_over_time()

                #Inserting data into dataframe 
                df_search_main.insert(col_length, (geo_code_index +'_'+ rel_query), rel_query_over_time_search[rel_query], True)
                row_length, col_length = df_search_main.shape

        return df_search_main, search_queries
    

In [3]:
df_search_cad, search_queries_cad  = search_index('beyond meat',['CA']).df_search_tiator()

#Adding time dealay otherwise google trend breaks when pulling too many series
time.sleep(10)
df_search_us, search_queries_us  = search_index('beyond meat',['US']).df_search_tiator()

In [9]:
time.sleep(10)
#Custom search term
df_search_custom, search_queries_custom  = search_index(["vegetarian",'salad bar',"vegetarian recipe"],['US','US-CT','CA','CA-ON']).df_search_tiator()

In [11]:
date_index_2019 = 180

with pd.ExcelWriter('search_data.xlsx') as writer:  
    df_search_cad[date_index_2019:].to_excel(writer, sheet_name='Sheet_name_1')
    df_search_us[date_index_2019:].to_excel(writer, sheet_name='Sheet_name_2')
    df_search_custom[date_index_2019:].to_excel(writer, sheet_name='Sheet_name_3')